In [ ]:
from Lab_Analyses.Utilities.get_dFoF import get_dFoF
from Lab_Analyses.Utilities.load_mat_files import load_mat

In [ ]:
SnFr_path = r"Z:\People\Jake\Imaging_Data\JW033\220103\dendrite_1\summed"
SnFr_file = "JW033_0103_00001_00001_summed_50_Analyzed_ByJake"

SnFr_data = load_mat(SnFr_file, SnFr_file, SnFr_path)

In [ ]:
SnFr_fluoro = SnFr_data.Fluorescence_Measurement

In [ ]:
SnFr_sampling_rate = 58.30


In [ ]:
SnFr_fluoro[0]

In [ ]:
SnFr_dFoF, SnFr_PdFoF, SnFr_baseline = get_dFoF(SnFr_fluoro[0],SnFr_sampling_rate, smooth_window=0.5)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
plt.figure()
plt.plot(SnFr_dFoF)
plt.figure()
plt.plot(SnFr_PdFoF)
plt.figure()
plt.plot(SnFr_fluoro[0])
plt.plot(SnFr_baseline)

In [ ]:
import numpy as np

new_fluo = (SnFr_fluoro[0] - SnFr_fluoro[0].min()) / (SnFr_fluoro[0].max() - SnFr_fluoro[0].min())
new_base = (SnFr_baseline -  SnFr_fluoro[0].min()) / (SnFr_fluoro[0].max() - SnFr_fluoro[0].min())

In [ ]:
plt.figure()
plt.plot(new_fluo)
plt.plot(new_base)

In [ ]:
c = np.array([5,6,7,8]).reshape(1,-1)
a = np.append(a,c,axis=0)
a[1:,:]

In [ ]:
a = [np.array([1,2,3]), np.array([4,5,6])]
b = np.vstack(a).T
b

In [ ]:
b.mean(axis=1).reshape(-1,1)

In [ ]:
c = np.array([1,2,3]).reshape(-1,1)
d = np.array([4,5,6]).reshape(-1,1)
e = [c,d]
f = np.hstack(e).mean(axis=1).reshape(-1,1)

In [ ]:
file1 = "JW033_0103_00001_00001_corrected"
file2 = "JW033_0103_00001_00002_corrected"
file3 = "JW033_0103_00001_00003_corrected"
file4 = "JW033_0103_00002_00001_corrected"
file5 = "JW033_0103_00002_00002_corrected"
file6 = "JW033_0103_00002_00003_corrected"
files = [file1, file2, file3, file4, file5, file6]
frame_counts = [800,800,650,800,800,440]

In [ ]:
import re

name = "JW015_1113_00002_00001_summed_50"
a = re.search("[A-Z]{2}[0-9]{3}", name).group()
a

In [ ]:
import pickle
with open("viewer_test.pickle", "rb") as f:
    file = pickle.load(f)

In [ ]:
a = Object(name="Jake")

In [ ]:
from PyQt5 import QtCore, QtGui, QtWidgets
import sys
 
class Ui_MainWindow(QtWidgets.QWidget):
    def setupUi(self, MainWindow):
        MainWindow.resize(422, 255)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
 
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(160, 130, 93, 28))
 
        # For displaying confirmation message along with user's info.
        self.label = QtWidgets.QLabel(self.centralwidget)   
        self.label.setGeometry(QtCore.QRect(170, 40, 201, 111))
 
        # Keeping the text of label empty initially.      
        self.label.setText("")    
 
        MainWindow.setCentralWidget(self.centralwidget)
        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
 
    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.pushButton.setText(_translate("MainWindow", "Proceed"))
        self.pushButton.clicked.connect(self.takeinputs)
         
    def takeinputs(self):
        name, done1 = QtWidgets.QInputDialog.getText(
             self, 'Input Dialog', 'Enter your name:')
 
        roll, done2 = QtWidgets.QInputDialog.getInt(
           self, 'Input Dialog', 'Enter your roll:') 
 
        cgpa, done3 = QtWidgets.QInputDialog.getDouble(
              self, 'Input Dialog', 'Enter your CGPA:')
 
        langs =['C', 'c++', 'Java', 'Python', 'Javascript']
        lang, done4 = QtWidgets.QInputDialog.getItem(
          self, 'Input Dialog', 'Language you know:', langs)
 
        if done1 and done2 and done3 and done4 :
             # Showing confirmation message along
             # with information provided by user.
             self.label.setText('Information stored Successfully\nName: '
                                 +str(name)+'('+str(roll)+')'+'\n'+'CGPA: '
                                 +str(cgpa)+'\nSelected Language: '+str(lang))  
  
             # Hide the pushbutton after inputs provided by the use.
             self.pushButton.hide()     
               
              
              
if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show() 
 
    sys.exit(app.exec_())

In [ ]:
import numpy as np

ops = np.load("ops.npy", allow_pickle=True)

In [ ]:
ops